In [1]:
from google.colab import drive
drive.mount('./mydata')

Mounted at ./mydata


In [2]:
import sys
sys.path.append('./mydata/MyDrive/COQAR')
sys.path.append('./mydata/MyDrive/COQAR/rewriting')

In [3]:
!pip install transformers
!pip install rouge_metric
!pip install nltk==3.6.5
!pip install sentencepiece

     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 40.3 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 151 kB 5.2 MB/s 
     |████████████████████████████████| 1.5 MB 5.2 MB/s 
     |████████████████████████████████| 749 kB 47.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [4]:
import torch
import evaluation
import argparse
import qrdatasets
import models
from utils import *
import random
import t5small
import t5base
import os
import nltk

In [5]:
def get_arguments():
    parser = argparse.ArgumentParser(description = '')
    parser.add_argument('--action', default='train',choices=['eval', 'pre-eval', 'train'],
                        help='"pre-eval" evaluates the model on dev-sets, while "eval" evaluates on test sets.')
    parser.add_argument('--dataset_name', default='mixed',choices=['canard', 'coqar', 'mixed'],
                        help='Use this argument only when training. With "mixed", a mix of CANARD and COQAR is used.')
    parser.add_argument('--model_path', type=str, default='',
                        help='Only for eval/pre-eval. Path of the model to evaluate.')
    # parameters
    parser.add_argument('--epochs', default=3, type=int, help='only for training')
    parser.add_argument('--learning_rate', default=0.00005, type=float, help='only for training')
    parser.add_argument('--history_size', default=3, type=int, help='only for training')  # not only for training
    parser.add_argument('--batch_size', default=16, type=int)
    parser.add_argument('--dropout_rate', default=0.1, type=float, help='only for training')
    parser.add_argument('--include_story', default=True, help='include_story')
    parser.add_argument('--model_size', default='small', help='model_size')
    parser.add_argument('--smoke_test', dest='smoke_test', action='store_true', help='only for training')
    parser.set_defaults(smoke_test=False)
    args = parser.parse_args([])
    return args

In [6]:
random.seed(63254)
args = get_arguments()

# default parameters
hparams = {
    'epochs' : 3,
    'learning_rate' : 0.00005,
    'batch_size' : 16,
    'weight_decay' : 0.0,
    'history_size' : 20,
    'dropout_rate' : 0.1,
    'include_story' : True,
    'model_size' : 'small'
}
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
# overrides default params
if args.batch_size != None:
    hparams['batch_size'] = args.batch_size
if args.dropout_rate != None:
    hparams['dropout_rate'] = args.dropout_rate
if args.learning_rate != None:
    hparams['learning_rate'] = args.learning_rate
if args.history_size != None:
    hparams['history_size'] = args.history_size
if args.model_size != None:
    hparams['model_size'] = args.model_size

In [8]:
t5_models = [t5small, t5base]

if hparams['model_size'] == 'small':
  t5_model = t5_models[0]
elif hparams['model_size'] == 'base':
  t5_model = t5_models[1]

print('current t5 model:',t5_model)

current t5 model: <module 't5small' from './mydata/MyDrive/COQAR/rewriting/t5small.py'>


In [ ]:
# train the model

### check model.train_dev_loop stop condition, should use the second or third last epoch as trained model
args.action = 'train'
model_epoch_path = './mydata/MyDrive/COQAR/trained_models/training0/epoch1'
if args.action == 'train':
    dir_path = create_and_get_dir('./mydata/MyDrive/COQAR/trained_models', 'training')
    
    models.train_dev_loop(args, hparams, log_dir_path = dir_path, t5_model = t5_model, model_epoch_path = model_epoch_path)

    # models.train_dev_loop(args, hparams, log_dir_path = dir_path, t5_model = t5_model)

In [11]:
# evaluate the model
args.action = 'eval'
args.model_path = './mydata/MyDrive/COQAR/trained_models/training1/epoch11'


if args.action in ['eval', 'pre-eval']:
    
    model = torch.load(args.model_path)
    if args.action == 'eval':
        
        coqar = qrdatasets.get_coqar_test_set(hparams['include_story'])
        canard = qrdatasets.get_canard_test_set(hparams['include_story'])

    if args.action == 'pre-eval':
    
        _, coqar = qrdatasets.get_coqar_train_and_dev_sets(hparams['include_story'])
        canard = qrdatasets.get_canard_dev_set(hparams['include_story'])

    # data is formatted now
    for data, dataset_name in [(coqar, 'coqar'), (canard, 'canard')]:
        predictions = evaluation.generate_predictions(
            model = model,
            data = data,
            # make dataset in the same way as training
            dataset_maker = t5_model.make_dataset,
            output_tokenizer = t5_model.get_output_tokenizer(),
            hparams = hparams)

        print(f"{dataset_name}: {evaluation.evaluate(predictions, data['references'])}")
        #evaluation.display_examples(predictions, data['references'], data['input'])

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


coqar: {'BLEU': 0.40307622255484515, 'ROUGE': 0.4293164514423723, 'METEOR': 0.598848714608091}


Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors


canard: {'BLEU': 0.47249881382811804, 'ROUGE': 0.7052649439518603, 'METEOR': 0.7041165688362422}
